In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
import matplotlib.pyplot as plt

In [23]:
# Cargar los datos
df = pd.read_csv('titanic_MD.csv')
df_good = pd.read_csv('titanic.csv')

In [3]:
# 1. Reporte detallado de missing data para todas las columnas.
print(df.isnull().sum())

PassengerId     0
Survived        0
Pclass          0
Name            0
Sex             0
Age            25
SibSp           3
Parch          12
Ticket          0
Fare            8
Cabin           0
Embarked       12
dtype: int64


2.


*   Age: Se usara la Media porque el promedio de todos a bordo puede abarcar mejor el rango normal.
*   SibSp: Se usara la Moda porque son solo 3 datos faltantes y se tomaran todos por igual
*   Parch: Se usara la Media aproximada ya que son solo 12 datos.
*   Fare: Se usara la media, ya que tendremos la tarifa mas cercana a la real
*   Embarked: Se usara la moda, ya que hay pocos datos faltantes



In [9]:
df.replace(r'\?', np.nan, regex = True, inplace = True)
df.dropna().shape

(100, 12)

In [29]:
# a. Imputación general
imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer_median = SimpleImputer(missing_values=np.nan, strategy='median')

cambios = pd.DataFrame(imputer_mode.fit_transform(df[['Sex']]), columns = ['Sex1'])

#Age
cambios['Age_mean'] = imputer_mean.fit_transform(df[['Age']])
cambios['Age_mode'] = imputer_mode.fit_transform(df[['Age']])
cambios['Age_median'] = imputer_median.fit_transform(df[['Age']])

#SibSp
cambios['SibSp_mean'] = imputer_mean.fit_transform(df[['SibSp']])
cambios['SibSp_mode'] = imputer_mode.fit_transform(df[['SibSp']])
cambios['SibSp_median'] = imputer_median.fit_transform(df[['SibSp']])

#Parch
cambios['Parch_mean'] = imputer_mean.fit_transform(df[['Parch']])
cambios['Parch_mode'] = imputer_mode.fit_transform(df[['Parch']])
cambios['Parch_median'] = imputer_median.fit_transform(df[['Parch']])

#Fare
cambios['Fare_mean'] = imputer_mean.fit_transform(df[['Fare']])
cambios['Fare_mode'] = imputer_mode.fit_transform(df[['Fare']])
cambios['Fare_median'] = imputer_median.fit_transform(df[['Fare']])

#Embarked
cambios['Embarked_mode'] = imputer_mode.fit_transform(df[['Embarked']])

df_clean = df.dropna()
cambios

,Sex1,Age_mean,Age_mode,Age_median,SibSp_mean,SibSp_mode,SibSp_median,Parch_mean,Parch_mode,Parch_median,Fare_mean,Fare_mode,Fare_median,Embarked_mode
0,?,38.000000,38.0,38.0,1.000000,1.0,1.0,0.000000,0.0,0.0,71.283300,71.2833,71.2833,C
1,female,35.000000,35.0,35.0,1.000000,1.0,1.0,0.000000,0.0,0.0,53.100000,53.1000,53.1000,S
2,male,54.000000,54.0,54.0,0.000000,0.0,0.0,0.000000,0.0,0.0,51.862500,51.8625,51.8625,S
3,female,35.692532,24.0,35.5,1.000000,1.0,1.0,0.461988,0.0,0.0,16.700000,16.7000,16.7000,S
4,female,58.000000,58.0,58.0,0.461111,0.0,0.0,0.000000,0.0,0.0,26.550000,26.5500,26.5500,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,female,47.000000,47.0,47.0,1.000000,1.0,1.0,1.000000,1.0,1.0,78.959191,26.5500,56.9292,S
179,?,35.692532,24.0,35.5,0.000000,0.0,0.0,0.000000,0.0,0.0,5.000000,5.0000,5.0000,S
180,female,56.000000,56.0,56.0,0.000000,0.0,0.0,0.461988,0.0,0.0,83.158300,83.1583,83.1583,S
181,?,19.000000,19.0,19.0,0.000000,0.0,0.0,0.000000,0.0,0.0,30.000000,30.0000,30.0000,S


In [18]:
def regre(x,y):
    lm = LinearRegression()
    lm.fit(x,y)
    y_hat = lm.predict(x)
    error = (1/2*np.mean(y_hat - y ) **2)

    #creación de un data frame
    df_dict = { 'PassengerId':df_clean.dropna().PassengerId, 'y_hat': y_hat }
    df = pd.DataFrame(df_dict)
    return(lm)

In [30]:
#Age
age_lm = regre(df_clean.dropna().drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1), df_clean['Age'].dropna())
df_clean['Age_lm'] = age_lm.predict(df_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
df_clean['Age_lm']

#SibSp
SibSp_lm = regre(df_clean.dropna().drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1), df_clean['SibSp'].dropna())
df_clean['Sibsp_lm'] = SibSp_lm.predict(df_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
df_clean['Sibsp_lm']

#Parch
Parch_lm = regre(df_clean.dropna().drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1), df_clean['Parch'].dropna())
df_clean['Parch_lm'] = Parch_lm.predict(df_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
df_clean['Parch_lm']

#Fare
Fare_lm = regre(df_clean.dropna().drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1), df_clean['Fare'].dropna())
df_clean['Fare_lm'] = Fare_lm.predict(df_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
df_clean['Fare_lm']

<ipython-input-30-bbcc04d1c869>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Age_lm'] = age_lm.predict(df_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
<ipython-input-30-bbcc04d1c869>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Sibsp_lm'] = SibSp_lm.predict(df_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
<ipython-input-30-bbcc04d1c869>:13: SettingWithCopyWarning: 
A value is trying to be s

0       71.2833
1       53.1000
2       51.8625
5       13.0000
7      263.0000
         ...   
173     83.1583
174     39.4000
176     25.9292
177     50.4958
181     30.0000
Name: Fare_lm, Length: 136, dtype: float64

In [21]:
def detect_outliers_percentile(data, threshold_lower=0.05, threshold_upper=0.95):
    lower_bound = data.quantile(threshold_lower)
    upper_bound = data.quantile(threshold_upper)
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    return outliers

outliers_percentile_Fare = detect_outliers_percentile(df['Fare'])
outliers_percentile_SibSp = detect_outliers_percentile(df['SibSp'])
outliers_percentile_Age = detect_outliers_percentile(df['Age'])
outliers_percentile_Parch = detect_outliers_percentile(df['Parch'])

print("Outliers para la variable 'Fare':")
print(outliers_percentile_Fare)

print("\nOutliers para la variable 'SibSp':")
print(outliers_percentile_SibSp)

print("\nOutliers para la variable 'Age':")
print(outliers_percentile_Age)

print("\nOutliers para la variable 'Parch':")
print(outliers_percentile_Parch)

Outliers para la variable 'Fare':
7      263.0000
13     263.0000
34      10.4625
42      10.4625
46       0.0000
59     262.3750
71     263.0000
82       8.0500
86     263.0000
137    512.3292
142      7.6500
148      7.6500
153    512.3292
155    262.3750
169      0.0000
179      5.0000
Name: Fare, dtype: float64

Outliers para la variable 'SibSp':
7      3.0
13     3.0
30     2.0
40     2.0
59     2.0
71     3.0
115    2.0
125    2.0
155    2.0
Name: SibSp, dtype: float64

Outliers para la variable 'Age':
9      65.00
15     71.00
31      3.00
34      2.00
43     62.00
50     63.00
53      2.00
55      0.92
86     64.00
87      4.00
91     65.00
125     4.00
129    80.00
156    70.00
158     6.00
168    11.00
Name: Age, dtype: float64

Outliers para la variable 'Parch':
86    4.0
Name: Parch, dtype: float64


In [36]:
df_good[['Age']].describe()



,Age
count,183.000000
mean,35.674426
std,15.643866
min,0.920000
25%,24.000000
50%,36.000000
75%,47.500000
max,80.000000


In [37]:
df_good[['SibSp']].describe()


,SibSp
count,183.000000
mean,0.464481
std,0.644159
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,3.000000


In [38]:
df_good[['Parch']].describe()


,Parch
count,183.000000
mean,0.475410
std,0.754617
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,4.000000


In [39]:
df_good[['Fare']].describe()

,Fare
count,183.000000
mean,78.682469
std,76.347843
min,0.000000
25%,29.700000
50%,57.000000
75%,90.000000
max,512.329200


In [42]:
cambios[['Age_mean', 'Age_mode', 'Age_median']].describe()

,Age_mean,Age_mode,Age_median
count,183.000000,183.000000,183.00000
mean,35.692532,34.095191,35.66623
std,14.526959,15.074700,14.52711
min,0.920000,0.920000,0.92000
25%,25.000000,24.000000,25.00000
50%,35.692532,31.000000,35.50000
75%,46.500000,46.500000,46.50000
max,80.000000,80.000000,80.00000


In [41]:
cambios[['SibSp_mean', 'SibSp_mode', 'SibSp_median']].describe()

,SibSp_mean,SibSp_mode,SibSp_median
count,183.000000,183.000000,183.000000
mean,0.461111,0.453552,0.453552
std,0.640775,0.643459,0.643459
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000
max,3.000000,3.000000,3.000000


In [32]:
cambios[['Parch_mean', 'Parch_mode', 'Parch_median']].describe()

,Parch_mean,Parch_mode,Parch_median
count,183.000000,183.000000,183.000000
mean,0.461988,0.431694,0.431694
std,0.728173,0.737147,0.737147
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000
max,4.000000,4.000000,4.000000


In [44]:
cambios[['Fare_mean', 'Fare_mode', 'Fare_median']].describe()

,Fare_mean,Fare_mode,Fare_median
count,183.000000,183.000000,183.000000
mean,78.959191,76.668079,77.996131
std,75.314418,76.077057,75.449730
min,0.000000,0.000000,0.000000
25%,30.000000,26.550000,30.000000
50%,61.175000,55.441700,56.929200
75%,90.000000,90.000000,90.000000
max,512.329200,512.329200,512.329200


In [45]:
df_good[['Embarked']].value_counts()

Embarked
S           116
C            65
Q             2
dtype: int64

In [46]:
cambios[['Embarked_mode']].value_counts()

Embarked_mode
S                122
C                 59
Q                  2
dtype: int64

5. Comparaciones

La Media (mean) fue el metodo que mas se logro acercar a los datos reales para (Age, SibSp, Parch, Fare). El error de la media fue en todos los casos muy cercano por lo que no representaria ningun tipo de problema y los datos no estarian cesgados. Al igual para (Embarked) la cual falla por poco y no representa peligro alguno


6. Conclusiones:

Riesgo de Sesgo con la Moda: Al usar la moda para imputar valores faltantes, existe un riesgo significativo de sesgar la data, especialmente si la cantidad de información faltante es alta. Si la moda es la única opción viable y la información faltante es considerable, la eliminación de dichos registros puede ser preferible para evitar sesgar el modelo.

Uso de Medias y Medianas: Las medias y medianas son métodos eficientes para imputar datos rápidamente, dependiendo de la variabilidad y distribución de los datos. Identificar la distribución de los datos antes de elegir entre la media y la mediana puede ayudar a seleccionar el método más adecuado. También se debe considerar si es necesario explorar otras técnicas para imputar los valores faltantes, especialmente cuando la distribución de los datos no es normal.

Modelos de Regresión para Imputación: El uso de modelos de regresión para imputar valores faltantes puede ser efectivo cuando la cantidad de datos faltantes no es significativa y hay suficientes datos para entrenar el modelo. Sin embargo, si la cantidad de datos faltantes es considerable, los modelos de regresión pueden no ser la mejor opción, ya que la falta de información suficiente puede afectar negativamente el rendimiento del modelo.

Al considerar estas conclusiones, es importante evaluar cuidadosamente la cantidad de datos faltantes, la distribución de los datos y la viabilidad de cada método de imputación para garantizar la integridad y la precisión de los resultados.

In [49]:
scaler = StandardScaler()
sscale = cambios.copy()
for col in sscale.select_dtypes(include=['float', 'int']).columns:
    sscale[col+'_z'] = scaler.fit_transform(sscale[[col]])
sscale.head()

,Sex1,Age_mean,Age_mode,Age_median,SibSp_mean,SibSp_mode,SibSp_median,Parch_mean,Parch_mode,Parch_median,...,Age_median_z,SibSp_mean_z,SibSp_mode_z,SibSp_median_z,Parch_mean_z,Parch_mode_z,Parch_median_z,Fare_mean_z,Fare_mode_z,Fare_median_z
0,?,38.000000,38.0,38.0,1.000000,1.0,1.0,0.000000,0.0,0.0,...,0.161090,8.433032e-01,0.851565,0.851565,-6.361892e-01,-0.587235,-0.587235,-0.102198,-0.070975,-0.089215
1,female,35.000000,35.0,35.0,1.000000,1.0,1.0,0.000000,0.0,0.0,...,-0.045987,8.433032e-01,0.851565,0.851565,-6.361892e-01,-0.587235,-0.587235,-0.344292,-0.310642,-0.330875
2,male,54.000000,54.0,54.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,1.265501,-7.215893e-01,-0.706799,-0.706799,-6.361892e-01,-0.587235,-0.587235,-0.360768,-0.326953,-0.347322
3,female,35.692532,24.0,35.5,1.000000,1.0,1.0,0.461988,0.0,0.0,...,-0.011474,8.433032e-01,0.851565,0.851565,7.644262e-17,-0.587235,-0.587235,-0.828925,-0.790417,-0.814639
4,female,58.000000,58.0,58.0,0.461111,0.0,0.0,0.000000,0.0,0.0,...,1.541603,8.686898e-17,-0.706799,-0.706799,-6.361892e-01,-0.587235,-0.587235,-0.697781,-0.660588,-0.683730


In [52]:
scaler = MinMaxScaler()
min_max_s = cambios.copy()
for col in min_max_s.select_dtypes(include=['float', 'int']).columns:
    min_max_s[col+'_norm'] = scaler.fit_transform(min_max_s[[col]])
min_max_s.head()

,Sex1,Age_mean,Age_mode,Age_median,SibSp_mean,SibSp_mode,SibSp_median,Parch_mean,Parch_mode,Parch_median,...,Age_median_norm,SibSp_mean_norm,SibSp_mode_norm,SibSp_median_norm,Parch_mean_norm,Parch_mode_norm,Parch_median_norm,Fare_mean_norm,Fare_mode_norm,Fare_median_norm
0,?,38.000000,38.0,38.0,1.000000,1.0,1.0,0.000000,0.0,0.0,...,0.468892,0.333333,0.333333,0.333333,0.000000,0.0,0.0,0.139136,0.139136,0.139136
1,female,35.000000,35.0,35.0,1.000000,1.0,1.0,0.000000,0.0,0.0,...,0.430956,0.333333,0.333333,0.333333,0.000000,0.0,0.0,0.103644,0.103644,0.103644
2,male,54.000000,54.0,54.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.671219,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.101229,0.101229,0.101229
3,female,35.692532,24.0,35.5,1.000000,1.0,1.0,0.461988,0.0,0.0,...,0.437279,0.333333,0.333333,0.333333,0.115497,0.0,0.0,0.032596,0.032596,0.032596
4,female,58.000000,58.0,58.0,0.461111,0.0,0.0,0.000000,0.0,0.0,...,0.721801,0.153704,0.000000,0.000000,0.000000,0.0,0.0,0.051822,0.051822,0.051822


In [53]:
scaler = MaxAbsScaler()
maxscal = cambios.copy()
for col in maxscal.select_dtypes(include=['float', 'int']).columns:
    maxscal[col+'_MabsS'] = scaler.fit_transform(maxscal[[col]])
maxscal.head()

,Sex1,Age_mean,Age_mode,Age_median,SibSp_mean,SibSp_mode,SibSp_median,Parch_mean,Parch_mode,Parch_median,...,Age_median_MabsS,SibSp_mean_MabsS,SibSp_mode_MabsS,SibSp_median_MabsS,Parch_mean_MabsS,Parch_mode_MabsS,Parch_median_MabsS,Fare_mean_MabsS,Fare_mode_MabsS,Fare_median_MabsS
0,?,38.000000,38.0,38.0,1.000000,1.0,1.0,0.000000,0.0,0.0,...,0.47500,0.333333,0.333333,0.333333,0.000000,0.0,0.0,0.139136,0.139136,0.139136
1,female,35.000000,35.0,35.0,1.000000,1.0,1.0,0.000000,0.0,0.0,...,0.43750,0.333333,0.333333,0.333333,0.000000,0.0,0.0,0.103644,0.103644,0.103644
2,male,54.000000,54.0,54.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.67500,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.101229,0.101229,0.101229
3,female,35.692532,24.0,35.5,1.000000,1.0,1.0,0.461988,0.0,0.0,...,0.44375,0.333333,0.333333,0.333333,0.115497,0.0,0.0,0.032596,0.032596,0.032596
4,female,58.000000,58.0,58.0,0.461111,0.0,0.0,0.000000,0.0,0.0,...,0.72500,0.153704,0.000000,0.000000,0.000000,0.0,0.0,0.051822,0.051822,0.051822


Conclusiones:

Basándome en mis evaluaciones, la normalización a través del método de Standarization demostró ser altamente efectiva para mejorar la consistencia en la escala de los datos, lo que condujo a un comportamiento más eficiente en el conjunto de datos. Al observar los resultados, se notó que cuando los datos ya estaban normalizados o mostraban cierto grado de normalidad, los diversos métodos de normalización producían resultados comparables entre sí. Esta observación resalta la influencia significativa de la distribución y amplitud de los datos en la elección del método de normalización. Estos hallazgos resaltan la importancia de considerar detalladamente la naturaleza de los datos al determinar la estrategia de normalización más adecuada para el análisis de datos.